## Fine Tune BERT for Sentiment Analysis using IMDBDataset

Transformer architecture has encoder and decoder stack, hence called encoder-decoder architecture whereas BERT is just an encoder stack of transformer architecture. There are two variants, BERT-base and BERT-large, which differ in architecture complexity. The base model has 12 layers in the encoder whereas the Large has 24 layers.

BERT was trained on a large text corpus, which gives architecture/model the ability to better understand the language and to learn variability in data patterns and generalizes well on several NLP tasks. As it is bidirectional that means BERT learns information from both the left and the right side of a token’s context during the training phase

How to Fine-Tune BERT for Text Classification? (https://arxiv.org/pdf/1905.05583.pdf)

In [ ]:
!pip install -q transformers

In [ ]:
# Required to save models in HDF5 format

# !pip install pyyaml h5py

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer

#### Load IMDB Data

In [ ]:
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split = (tfds.Split.TRAIN, tfds.Split.TEST),
                                          as_supervised=True, with_info=True)

In [ ]:
ds_info

In [ ]:
len(ds_train), len(ds_test)

#### Explore IMDB Data

In [ ]:
type(ds_train)

In [ ]:
tfds.as_numpy(ds_train.take(5))

In [ ]:
for rec in tfds.as_numpy(ds_train.take(5)):
  print(rec)

In [ ]:
for review, label in tfds.as_numpy(ds_train.take(1)):
    print(review)
    print(type(review))
    print(review.decode())
    print(type(review.decode()))
    print(label)

In [ ]:
reviews = []
labels = []
for review, label in tfds.as_numpy(ds_train.take(5)):
    reviews.append(review.decode())
    labels.append(label)
    print(review.decode()[0:50], '\t', label)

In [ ]:
reviews[2]

In [ ]:
from transformers import pipeline
# classifier1 = pipeline("sentiment-analysis",'bert-base-uncased')
classifier = pipeline("sentiment-analysis",'distilbert-base-uncased-finetuned-sst-2-english')

In [ ]:
classifier(reviews[1])

#### Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Let’s prepare the data according to the format needed for the BERT model

- Input IDs – The input ids are often the only required parameters to be passed to the model as input. Token indices, numerical representations of tokens building the sequences that will be used as input by the model.

- Attention mask – Attention Mask is used to avoid performing attention on padding token indices. Mask value can be either 0 or 1, 1 for tokens that are NOT MASKED, 0 for MASKED tokens.

- Token type ids – It is used in use cases like sequence classification or question answering. As these require two different sequences to be encoded in the same input IDs. Special tokens, such as the classifier[CLS] and separator[SEP] tokens are used to separate the sequences.

In [ ]:

# The encode_plus  function of the tokenizer class will tokenize the raw input, add the special tokens,
# and pad the vector to a size equal to max length (that we can set).
def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

In [ ]:
# can be up to 512 for BERT
max_length = 512
batch_size = 6

In [ ]:
for review, label in tfds.as_numpy(ds_train.take(1)):
    encodedReview = convert_example_to_feature(review.decode())
    print(type(encodedReview))
    print(encodedReview.keys())
    print(encodedReview['input_ids'])
    print(encodedReview['token_type_ids'])
    print(encodedReview['attention_mask'])
    # print(encodedReview)
    print(review.decode())
    print(label)

#### Encode data

In [ ]:
# helper functions will help us to transform our raw data to an appropriate format ready to feed into the BERT model
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
  if (limit > 0):
      print("Using", limit, "Records from ds")
      ds = ds.take(limit)
  else:
      print("Using all Records from ds")
  for review, label in tfds.as_numpy(ds):
      bert_input = tokenizer.encode_plus(review.decode(),add_special_tokens = True,max_length = max_length,pad_to_max_length = True,
                                          return_attention_mask = True,)
      # bert_input = convert_example_to_feature(review.decode())
      input_ids_list.append(bert_input['input_ids'])
      token_type_ids_list.append(bert_input['token_type_ids'])
      attention_mask_list.append(bert_input['attention_mask'])
      label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


- **batch** - Combines consecutive elements of this dataset into batch
- **shuffle** - Randomly shuffles the elements of this dataset. This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements.

In [ ]:
# # train dataset
# ds_train_encoded = encode_examples(ds_train).shuffle(10000).batch(batch_size)
# # test dataset
# ds_test_encoded = encode_examples(ds_test).batch(batch_size)

# train dataset
ds_train_encoded = encode_examples(ds_train, limit=5000).shuffle(2000).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(ds_test, limit=1000).batch(batch_size)

#### Understandng ds_train_encoded structure

In [ ]:
for record in ds_train_encoded.take(1).as_numpy_iterator():
    print(type(record))
    # print(len(record))

In [ ]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(type(inputs))
    print(type(labels))

In [ ]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(inputs.keys())
    print(len(labels))

In [ ]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(len(inputs['input_ids']))
    print(len(inputs['token_type_ids']))
    print(len(inputs['attention_mask']))
    print(len(labels))

**As we have selected batch_size=6, we see each data record consists of 6 encoded reviews & 6 labels**

In [ ]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(inputs['input_ids'][0])
    print(inputs['token_type_ids'][0])
    print(inputs['attention_mask'][0])
    print(labels[0])

#### Link google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

#### Creating and training (fine tuning) the model

In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
# to avoid catastrophic forgetting (https://arxiv.org/pdf/1905.05583.pdf -- How to Fine-Tune BERT for Text Classification?)
learning_rate = 2e-5
# we will do just 1 epoch, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1
# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
# choosing Adam optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate, epsilon=1e-08)
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
to_train = True

In [ ]:
# The model will take around two hours on GPU to complete training, with just 1 epoch we can achieve over 93% accuracy on validation
# you can further increase the epochs and play with other parameters to improve the accuracy.

# Training takes about 12 to 15 mins for 5000 training and 1000 testing data rows, and one epoch
if to_train:
    bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)
    # model.save_weights('fineTuneBERTwithIMDB_weights.ckpt')
    # !cp fineTuneBERTwithIMDB_weights.* /mydrive

#### Use previously trained weights (instead of training)

In [ ]:
if not to_train:
    !cp /mydrive/fineTuneBERT/fineTuneBERTwithIMDB_weights.* /content

In [ ]:
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
model.load_weights('fineTuneBERTwithIMDB_weights.ckpt')

#### Inference on random sample

In [ ]:
# myreview = "This is a really good movie. I loved it and will watch again"
#myreview = "There was too much violence and I would suggest to avoid"
myreview = "Though the movie was good, it was a bit too long"
encoded_myreview = tokenizer.encode(myreview, truncation=True, padding=True, return_tensors="tf")

In [ ]:
tf_output = model.predict(encoded_myreview)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive'] #(0:negative, 1:positive)
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

In [ ]:
print(tf_output)
print(tf_prediction)

#### Inference on Test data

In [ ]:
reviews = []
labels = []
for review, label in tfds.as_numpy(ds_test.take(5)):
    reviews.append(review.decode())
    labels.append(label)
    print(review.decode()[0:50], '\t', label)

In [ ]:
i = 0
reviews[i]

In [ ]:
# myreview = "This is a really good movie. I loved it and will watch again"
myreview = reviews[i]
encoded_myreview = tokenizer.encode(myreview, truncation=True, padding=True, return_tensors="tf")

In [ ]:
tf_output = model.predict(encoded_myreview)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive'] #(0:negative, 1:positive)
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

In [ ]:
print(tf_output)
print(tf_prediction)

In [ ]:
classifier(myreview)

tokenizer. encode will encode our test example into integers using Bert tokenizer, then we use predict method on the encoded input to get our predictions. The model. predict will return logits, on which we can apply softmax function to get the probabilities for each class, and then using TensorFlow argmax function we can get the class with the highest probability and map it to text labels (positive or negative).

### Extra code -- to understand how tf.data.Dataset.from_tensor_slices() works

#### tf.data.Dataset.from_tensor_slices()
https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
  print(element)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.map(lambda x: x*2)
list(dataset.as_numpy_iterator())

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset.as_numpy_iterator():
  print(element)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
print(list(dataset.as_numpy_iterator()))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices({'a': ([1, 2], [3, 4]),
                                              'b': [5, 6]})
list(dataset.as_numpy_iterator()) == [{'a': (1, 3), 'b': 5},
                                      {'a': (2, 4), 'b': 6}]

In [ ]:
list(dataset.as_numpy_iterator())